Ce notebook utilise un modèle glove 6B préentrainé sur Wikipedia 2014 et Gigaword 5 (6B tokens, 400K vocab, uncased, 50d, 100d, 200d, & 300d vectors, 822 MB download) téléchargé ici: http://nlp.stanford.edu/data/glove.6B.zip
Notebook inspiré de https://www.kaggle.com/jhoward/improved-lstm-baseline-glove-dropout/notebook

In [16]:
import sys, os, re, csv, codecs 
import numpy as np
import pandas as pd, configparser

# keras
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences

from keras.layers import Dense, Input, LSTM, GlobalMaxPool1D, Bidirectional, Embedding, Dropout
from keras.models import Model

# sklearn
from sklearn.metrics import accuracy_score, confusion_matrix, classification_report

In [7]:
config = configparser.ConfigParser()
config.read('../config.cfg')
EMBEDDING_FILE=config['FILES']['GLOVE']
TRAIN_DATA_FILE=config['FILES']['TRAIN']
TEST_DATA_FILE=config['FILES']['TEST']

In [3]:
# Chargement des données sous forme de dataframes pandas
train = pd.read_csv(TRAIN_DATA_FILE)
test = pd.read_csv(TEST_DATA_FILE)

# Définition des différents labels disponibles sous forme de liste
list_classes = ["toxic", "severe_toxic", "obscene", "threat", "insult", "identity_hate"]

y = train[list_classes].values
list_sentences_train = train["comment_text"].fillna("_na_").values
list_sentences_test = test["comment_text"].fillna("_na_").values

In [5]:
embed_size = 300 # taille du vecteur
max_features = 20000 # nombre de mots uniques utilisés (i.e nombre de lignes dans le vecteur d'embedding)
maxlen = 100 # nombre maximum de mots à considérer dans un commentaire

In [6]:
tokenizer = Tokenizer(num_words=max_features)
tokenizer.fit_on_texts(list(list_sentences_train))
list_tokenized_train = tokenizer.texts_to_sequences(list_sentences_train)
list_tokenized_test = tokenizer.texts_to_sequences(list_sentences_test)
X_t = pad_sequences(list_tokenized_train, maxlen=maxlen)
X_te = pad_sequences(list_tokenized_test, maxlen=maxlen)

In [8]:
def get_coefs(word,*arr): 
    return word, np.asarray(arr, dtype='float32')
embeddings_index = dict(get_coefs(*o.strip().split()) for o in open(EMBEDDING_FILE))

In [9]:
word_index = tokenizer.word_index
# le modèle pré-entrainé contient 3 millions de mots contre 20000 pour notre jeu de données.
vocabulary_size=min(len(word_index)+1,max_features)

# la matrice d'embedding représente le vocabulaire final à utiliser. 
# On ne gardera que 20000 vecteurs du modèle pré-entrainé. Leur longueur est de 300.
embedding_matrix = np.zeros((vocabulary_size, embed_size))


for word, i in word_index.items():
    if i>=max_features:
        continue
    try:
        embedding_vector = embeddings_index[word]
        if len(embedding_vector) == 300:
            embedding_matrix[i] = embedding_vector
        elif len(embedding_vector) > 300:
            embedding_matrix[i] = embedding_vector[0:300]
        else:
            while len(embedding_vector) < 300:
                embedding_vector.append(0)
    except KeyError:
        vec = np.zeros(embed_size)
        embedding_matrix[i]=vec

In [10]:
inp = Input(shape=(maxlen,))
x = Embedding(max_features, embed_size, weights=[embedding_matrix])(inp)
x = Bidirectional(LSTM(50, return_sequences=True, dropout=0.1, recurrent_dropout=0.1))(x)
x = GlobalMaxPool1D()(x)
x = Dense(50, activation="relu")(x)
x = Dropout(0.1)(x)
x = Dense(6, activation="sigmoid")(x)
model = Model(inputs=inp, outputs=x)
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

In [11]:
model.fit(X_t, y, batch_size=32, epochs=2, validation_split=0.1)

Train on 143613 samples, validate on 15958 samples
Epoch 1/2
143613/143613 [==============================] - 1134s 8ms/step - loss: 0.0546 - acc: 0.9806 - val_loss: 0.0459 - val_acc: 0.9830
Epoch 2/2
143613/143613 [==============================] - 1090s 8ms/step - loss: 0.0407 - acc: 0.9844 - val_loss: 0.0459 - val_acc: 0.9828


In [12]:
test_label = pd.read_csv(config['FILES']['LABEL'])
test_label_strip = test_label[test_label.toxic != -1]
yt = test_label_strip[list_classes].values

In [13]:
model.evaluate(X_te[test_label_strip.index], yt, batch_size=1024)

63978/63978 [==============================] - 70s 1ms/step


[0.06264560487349713, 0.9744313591276889]

In [14]:
y_pred_t = model.predict(X_te, batch_size=1024)

In [17]:
print(accuracy_score(yt, y_pred_t[test_label_strip.index].round()))

0.8918847103691894


In [18]:
# Classification pour chaque label de façon indépendante
for label in range(0,6):
    print('... Traitement du label {}'.format(list_classes[label]))
    # On calcule l'accuracy des prédictions
    acc = accuracy_score(yt[:,label], y_pred_t[test_label_strip.index, label].round())
    print('L\'accuracy du jeu test est {}'.format(acc))
    print('Matrice de confusion sur le jeu test:')
    print(confusion_matrix(yt[:,label], y_pred_t[test_label_strip.index, label].round()))
    print('Rapport : ')
    print(classification_report(yt[:,label], y_pred_t[test_label_strip.index, label].round()) )

... Traitement du label toxic
L'accuracy du jeu test est 0.9332114164243959
Matrice de confusion sur le jeu test:
[[54891  2997]
 [ 1276  4814]]
Rapport : 
              precision    recall  f1-score   support

           0       0.98      0.95      0.96     57888
           1       0.62      0.79      0.69      6090

   micro avg       0.93      0.93      0.93     63978
   macro avg       0.80      0.87      0.83     63978
weighted avg       0.94      0.93      0.94     63978

... Traitement du label severe_toxic
L'accuracy du jeu test est 0.9926693550908124
Matrice de confusion sur le jeu test:
[[63327   284]
 [  185   182]]
Rapport : 
              precision    recall  f1-score   support

           0       1.00      1.00      1.00     63611
           1       0.39      0.50      0.44       367

   micro avg       0.99      0.99      0.99     63978
   macro avg       0.69      0.75      0.72     63978
weighted avg       0.99      0.99      0.99     63978

... Traitement du label obs